# 🧬 Genetic Trait Inheritance Estimator
This notebook estimates the likelihood of inheriting parent traits using phenotype-gene associations, genetic linkage, and user-reported traits — no DNA testing required.

# Updated to compare Parent and Child answers

## 🔹 Step 1: Enter a Parent Trait
This is a trait your parent has, like `freckles`, `lactose intolerance`, or a known condition.
We'll use it to look up genes associated with the trait.


In [17]:
import requests
import json
import emoji
from collections import defaultdict


ENSEMBL_API = "https://rest.ensembl.org"
SPECIES = "homo_sapiens"

In [18]:
def get_associated_genes(phenotype):
    url = f"{ENSEMBL_API}/phenotype/term/{SPECIES}/{phenotype}?content-type=application/json"
    response = requests.get(url)
    if response.status_code != 200:
        return set(), []
    data = response.json()
    genes = set()
    for entry in data:
        if "attributes" in entry and "associated_gene" in entry["attributes"]:
            genes.update(g.strip() for g in entry["attributes"]["associated_gene"].split(","))
    return genes, data

## 🔹 Step 2: Find Genes Linked to the Trait Gene(s)
We now expand the search to genes nearby (linked) to those associated with the original trait.


In [19]:
# Get genes nearby a target gene

def get_linked_traits(genes):
    traits = defaultdict(set)
    for gene in genes:
        url = f"{ENSEMBL_API}/phenotype/gene/{SPECIES}/{gene}?content-type=application/json"
        r = requests.get(url)
        if r.status_code != 200:
            print(f"⚠️ Could not locate {gene} in Ensembl")
            continue
        gene_data = r.json()
        for record in gene_data:
            trait = record.get("description")
            if trait:
                traits[trait].add(gene)
    return traits

## 🔹 Step 3: Survey the User
Ask if the user has any traits associated with the linked genes.


In [23]:
def ask_trait_survey(trait_list, identity_label):
    answers = {}
    print(f"\n👤 {identity_label}, please answer the following (yes / no / unknown):")
    for trait in trait_list:
        while True:
            ans = input(f"👉 {identity_label} — Do you have this trait? '{trait}'? ").strip().lower()
            if ans in ["yes", "no", "unknown"]:
                answers[trait] = ans
                break
            else:
                print("❗Please answer with 'yes', 'no', or 'unknown'.")
    return answers


def compare_traits(parent, child):
    yes_yes = sum(1 for trait in parent if parent[trait] == 'yes' and child.get(trait) == 'yes')
    no_no = sum(1 for trait in parent if parent[trait] == 'no' and child.get(trait) == 'no')
    total = len(parent)
    return yes_yes, no_no, total

## 🔹 Step 5: Run It All Together
This pulls everything together into a single run.

In [24]:
def main():
    trait_input = input("👩‍👧 Enter a trait your parent has (e.g., 'freckles', 'lactose intolerance'): ").strip().lower()

    # Step 1: Find genes
    parent_genes, _ = get_associated_genes(trait_input)
    print(f"\n🧬 Genes associated with '{trait_input}':", ", ".join(sorted(parent_genes)))

    # Step 2: Find linked traits
    linked_traits = get_linked_traits(parent_genes)
    trait_list = sorted(linked_traits.keys())[:30]  # limit for brevity
    print(f"\n🔗 Found {len(trait_list)} linked traits.")

    # Step 3: Get parent and child answers
    parent_answers = ask_trait_survey(trait_list, "Parent")
    child_answers = ask_trait_survey(trait_list, "You")

    # Step 4: Compare
    yes_yes, no_no, total = compare_traits(parent_answers, child_answers)
    print(f"\n🎯 Survey Complete!")
    print(f"Shared expressed traits (yes-yes): {yes_yes}")
    print(f"Shared non-traits (no-no): {no_no}")
    print(f"Total traits compared: {total}")

In [25]:
main()


👩‍👧 Enter a trait your parent has (e.g., 'freckles', 'lactose intolerance'):  freckles



🧬 Genes associated with 'freckles': AKAP1, ASIP, BNC2, FANCA, HERC2, HSPA12A, IRF4, LOC105375983, LOC107984363, MC1R, MSI2, OCA2, PPARGC1B, RAB11FIP2, RP11-62F24.2, SEC5L1, SLC45A2, SPATA33, TYR
⚠️ Could not locate LOC105375983 in Ensembl
⚠️ Could not locate LOC107984363 in Ensembl
⚠️ Could not locate AKAP1 in Ensembl

🔗 Found 30 linked traits.

👤 Parent, please answer the following (yes/no):


👉 Parent — Do you have this trait? 'ALBINISM, OCULOCUTANEOUS, TYPE IA'? (yes/no):  yes
👉 Parent — Do you have this trait? 'ALBINISM, OCULOCUTANEOUS, TYPE IB'? (yes/no):  yes
👉 Parent — Do you have this trait? 'ALBINISM, OCULOCUTANEOUS, TYPE II'? (yes/no):  yes
👉 Parent — Do you have this trait? 'ALBINISM, OCULOCUTANEOUS, TYPE IV'? (yes/no):  yes
👉 Parent — Do you have this trait? 'Acute lymphoblastic leukemia'? (yes/no):  yes
👉 Parent — Do you have this trait? 'Acute myeloid leukemia'? (yes/no):  yes
👉 Parent — Do you have this trait? 'Adenoid Cystic Breast Carcinoma'? (yes/no):  yes
👉 Parent — Do you have this trait? 'Adrenal Gland Neuroblastoma'? (yes/no):  yes
👉 Parent — Do you have this trait? 'Albinism oculo-cutaneous type 2'? (yes/no):  yes
👉 Parent — Do you have this trait? 'Albinism oculocutaneous type 4'? (yes/no):  yes
👉 Parent — Do you have this trait? 'Alveolar rhabdomyosarcoma'? (yes/no):  yes
👉 Parent — Do you have this trait? 'Ampulla of Vater Carcinoma'? (yes/no):  yes



👤 You, please answer the following (yes/no):


👉 You — Do you have this trait? 'ALBINISM, OCULOCUTANEOUS, TYPE IA'? (yes/no):  yes
👉 You — Do you have this trait? 'ALBINISM, OCULOCUTANEOUS, TYPE IB'? (yes/no):  yes
👉 You — Do you have this trait? 'ALBINISM, OCULOCUTANEOUS, TYPE II'? (yes/no):  yes
👉 You — Do you have this trait? 'ALBINISM, OCULOCUTANEOUS, TYPE IV'? (yes/no):  yes
👉 You — Do you have this trait? 'Acute lymphoblastic leukemia'? (yes/no):  yes
👉 You — Do you have this trait? 'Acute myeloid leukemia'? (yes/no):  yes
👉 You — Do you have this trait? 'Adenoid Cystic Breast Carcinoma'? (yes/no):  yes
👉 You — Do you have this trait? 'Adrenal Gland Neuroblastoma'? (yes/no):  yes
👉 You — Do you have this trait? 'Albinism oculo-cutaneous type 2'? (yes/no):  yes
👉 You — Do you have this trait? 'Albinism oculocutaneous type 4'? (yes/no):  yes
👉 You — Do you have this trait? 'Alveolar rhabdomyosarcoma'? (yes/no):  yes
👉 You — Do you have this trait? 'Ampulla of Vater Carcinoma'? (yes/no):  yes
👉 You — Do you have this trait? 'Ana


🎯 Survey Complete!
Shared expressed traits (yes-yes): 28
Shared non-traits (no-no): 0
Total traits compared: 30
